# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [107]:
# your code here
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('tic-tac-toe.csv')

print(df.head())



  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True


In [108]:
print(df.head())
print(df.info())
print(df.isnull().sum())

  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB
None
TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64


In [109]:
# Apply explicit mapping for grid features
# Map 'x' -> 1, 'o' -> -1, 'b' -> 0
value_map = {'x': 1, 'o': -1, 'b': 0}
for col in df.columns[:-1]:  # Skip the last column which is the target ('class')
    df[col] = df[col].map(value_map)

In [110]:
X = df.drop(columns=['class'])
y = df['class']

In [111]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the first few rows to verify
print(x_train.head())
print(y_train.head())

     TL  TM  TR  ML  MM  MR  BL  BM  BR
302  -1   1   1  -1   1  -1   0   1   0
467  -1   0  -1  -1   1   0   1   1   1
294   1   0   0   0   1   0  -1  -1   1
548   0  -1   1  -1   0   1   0   0   1
465  -1   0  -1   1  -1   0   1   1   1
302    True
467    True
294    True
548    True
465    True
Name: class, dtype: bool


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [112]:
# your code here
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import models

Step 1: Split the Traning and test Data

In [113]:
model = models.Sequential()

model.add(layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))




Step 2: Compile the model

In [114]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [118]:
# your code here
history = model.fit(x_train, y_train, epochs=26, batch_size=16, validation_split=0.2)


Epoch 1/20
31/31 [==============================] - 5s 155ms/step - loss: 0.0098 - accuracy: 1.0000 - val_loss: 0.0443 - val_accuracy: 0.9870
Epoch 2/20
31/31 [==============================] - 5s 152ms/step - loss: 0.0091 - accuracy: 1.0000 - val_loss: 0.0409 - val_accuracy: 0.9870
Epoch 3/20
31/31 [==============================] - 2s 74ms/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 0.0415 - val_accuracy: 0.9870
Epoch 4/20
31/31 [==============================] - 2s 81ms/step - loss: 0.0073 - accuracy: 1.0000 - val_loss: 0.0390 - val_accuracy: 0.9870
Epoch 5/20
31/31 [==============================] - 2s 49ms/step - loss: 0.0066 - accuracy: 1.0000 - val_loss: 0.0393 - val_accuracy: 0.9870
Epoch 6/20
31/31 [==============================] - 1s 30ms/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 0.0383 - val_accuracy: 0.9870
Epoch 7/20
31/31 [==============================] - 2s 51ms/step - loss: 0.0058 - accuracy: 1.0000 - val_loss: 0.0373 - val_accuracy: 0.9870
Epoch 8/20


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [116]:
# your code here
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {test_accuracy:.4f}')

6/6 [==============================] - 0s 54ms/step - loss: 0.0615 - accuracy: 0.9792
Test Accuracy: 0.9792


**Which approach(es) did you find helpful to improve your model performance?**

In [117]:
# your answer here
#THe use of ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.